In [ ]:
import avapi
import avstack
import pandas
import pickle

import os
from tqdm import tqdm
import numpy as np
import shutil

from agents import EgoCameraPerception, AgentsCameraPerception

## Define Perception/Tracking Models

In [ ]:
perception = {
    'ego':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
            model='fasterrcnn', dataset='carla', epoch='latest', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(
            model='pointpillars', dataset='carla', epoch='latest', threshold=0.2)
    },
    'agent':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
            model='fasterrcnn', dataset='carla-infrastructure', epoch='latest', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(
            model='pointpillars', dataset='carla-infrastructure', epoch='latest', threshold=0.2)
    },
    'joint':{
        'camera':None,
        'lidar':None
    }
}

perception_reversed = {
    'ego':{
        'camera':perception['agent']['camera'],
        'lidar':perception['agent']['lidar']
    },
    'agent':{
        'camera':perception['ego']['camera'],
        'lidar':perception['agent']['lidar']
    }
}

In [ ]:
# string is: algorithm-train_data-test_data
agents = [
    ('Camera -- Train: Vehicle;  Test: Vehicle',     EgoCameraPerception(perception),             'ego',   'ego'),
    ('Camera -- Train: Infra.;   Test: Infra',       AgentsCameraPerception(perception),          'agent', 'agent'),
    ('Camera -- Train: Vehicle.; Test: Infra',       AgentsCameraPerception(perception_reversed), 'ego',   'agent'),
    ('LiDAR  -- Train: Vehicle;  Test: Vehicle',     EgoLidarPerception(perception),              'ego',   'ego'),
    ('LiDAR  -- Train: Infra.;   Test: Infra',       AgentsLidarPerception(perception),           'agent', 'agent'),
    ('LiDAR  -- Train: Vehicle.; Test: Infra',       AgentsLidarPerception(perception_reversed),  'ego',   'agent')
]

## Test Perception Application

In [ ]:
data_dir_ego = '/data/spencer/CARLA/ego-lidar/'
data_dir_infra = '/data/spencer/CARLA/multi-agent-v1/'

SM_ego = avapi.carla.CarlaScenesManager(data_dir_ego)
SM_infra = avapi.carla.CarlaScenesManager(data_dir_infra)

In [ ]:
out_dir = 'application_results'
n_scenes = 10
n_frames = 50

# perform tests for each model
results = {}
print('Testing {} models'.format(len(agents)))
for i_model, (model_str, model, train, test) in enumerate(agents):
    results[model_str] = []
    
    # -- get test data loader
    if test == 'ego':
        SM = SM_ego
        vehicle = 'ego'
    elif test == 'agent':
        SM = SM_infra
        vehicle = 'agent'
    else:
        raise NotImplementedError(test)

    # -- determine data input type
    if 'camera' in model_str.lower():
        data_type = 'camera'
    else:
        data_type = 'lidar'

    # -- make save folders
    out_folder = os.path.join(out_dir, model_str)
    gt_dir = os.path.join(out_folder, 'ground-truth')
    det_dir = os.path.join(out_folder, 'detection-results')
    if os.path.exists(gt_dir):
        shutil.rmtree(gt_dir)
    if os.path.exists(det_dir):
        shutil.rmtree(det_dir)
    os.makedirs(gt_dir, exist_ok=False)
    os.makedirs(det_dir, exist_ok=False)
        
    # -- execute the model over the scene data
    n_scenes_run = min(n_scenes, len(SM.splits_scenes['val']))
    print('Running over {} scenes for model #{}/{} - {}'.format(n_scenes_run, i_model+1, len(agents), model_str))
    for i_scene, scene in enumerate(tqdm(SM.splits_scenes['val'][:n_scenes_run], position=0, leave=True)):
        SD = SM.get_scene_dataset_by_name(scene)

        # -- get all the frames for applicable sensors
        if data_type == 'camera':
            if vehicle == 'ego':
                sensors = [sensor for sensor in SD.sensor_IDs if
                               ('cam' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' not in sensor.lower())]
            else:
                sensors = [sensor for sensor in SD.sensor_IDs if 
                               ('cam' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' in sensor.lower())]
        else:
            if vehicle == 'ego':
                sensors = [sensor for sensor in SD.sensor_IDs if
                               ('lidar' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' not in sensor.lower())]
            else:
                sensors = [sensor for sensor in SD.sensor_IDs if 
                               ('lidar' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' in sensor.lower())]
                
        # -- loop sensors that apply
        for sensor in sensors:
            frames = SD.get_frames(sensor=sensor)
            # -- loop frames
            nframes_min = min(n_frames, len(frames))
            for frame in frames[:nframes_min]:                
                # -- get perception data
                try:
                    if data_type == 'camera':
                        data = SD.get_image(frame=frame, sensor=sensor)
                    else:
                        data = SD.get_lidar(frame=frame, sensor=sensor)
                        if 'infra' in sensor.lower():
                            data = data.transform_to_ground()
                    objs = SD.get_objects(frame=frame, sensor=sensor)
                except (FileNotFoundError, KeyError):
                    continue

                # -- determine if ego or agent data
                if vehicle == 'ego':
                    ego_data = data
                    agents_data = []
                else:
                    ego_data = None
                    agents_data = [data]

                # -- run perception model
                if data_type == 'camera':
                    outputs = model(ego_image=ego_data, agents_images=agents_data)
                else:
                    outputs = model(ego_pc=ego_data, agents_pcs=agents_data)

                # -- HACK: if it's an agent model, we are just considering one sensor at a time
                if vehicle != 'ego':
                    outputs = outputs[0]
                    
                # -- aggregate results
                truths = avstack.datastructs.DataContainer(
                    frame=frame,
                    timestamp=0.0,
                    data=[obj.box.project_to_2d_bbox(data.calibration) if data_type == 'camera' else obj.box for obj in objs],
                    source_identifier='truths',
                )
                res = avapi.evaluation.ResultManager(
                    idx=frame,
                    detections=outputs,
                    truths=truths,
                    metric='2D_IoU' if data_type == 'camera' else '3D_IoU',
                    threshold=0.5 if data_type == 'camera' else 0.3,
                    assign_algorithm='greedy',
                )
                results[model_str].append(res)

# save last results since it takes long AF...
out_file = 'perception_application_results.p'
with open(out_file, 'wb') as f:
    pickle.dump(results, f)
print('Just saved resutls to {}'.format(out_file))

## Analyze Results

In [ ]:
# load results
if 'results' not in locals():
    print('loading perception results...')
    out_file = 'perception_application_results.p'
    with open(out_file, 'rb') as f:
        results = pickle.load(f)
    print('Just loaded resutls from {}'.format(out_file))
else:
    print('Results variable seems to already be defined...keep going!')

In [ ]:
figs_out = 'perception_application_figures'
os.makedirs(figs_out, exist_ok=True)

# Separate into two for camera and lidar
ap_wrapped_by_sensor = {'camera':[], 'lidar':[]}
for model_str in results:
    ap, _, _ = avapi.evaluation.metrics.get_ap_from_results(results[model_str], by_class=True)    
    if 'camera' in model_str.lower():    ap_wrapped_by_sensor['camera'].append((model_str, ap))
    else:
        ap_wrapped_by_sensor['lidar'].append((model_str, ap))

# Make plots
for sensor, ap_wrapped in ap_wrapped_by_sensor.items():
    print('For sensor {}'.format(sensor))
    fig = avapi.evaluation.metrics.barplot_ap(ap_wrapped, reverse_subbars=True)
    fig.savefig(os.path.join(figs_out, f'barplot_ap_{sensor}.pdf'), format='pdf', bbox_inches="tight")

## Visualize Results

In [ ]:
data_dir_ego = '/data/spencer/CARLA/ego-lidar/'
data_dir_infra = '/data/spencer/CARLA/multi-agent-v1/'

SM_ego = avapi.carla.CarlaScenesManager(data_dir_ego)
SM_infra = avapi.carla.CarlaScenesManager(data_dir_infra)

In [ ]:
def show_model_on_sd(SM, sensor, models, idx_val=0, idx_frame=10):
    SD = SM.get_scene_dataset_by_name(SM.splits_scenes['val'][idx_val])
    img = SD.get_image(frame=SD.get_frames(sensor=sensor)[idx_frame], sensor=sensor)
    for name, model in models:
        dets = model(img)
        print(f'Showing model {name}')
        avapi.visualize.snapshot.show_image_with_boxes(img, dets, inline=True)

In [ ]:
# show testing on ego
thresh = 0.7
perception['ego']['camera'].threshold   = thresh
perception['agent']['camera'].threshold = thresh

models = [('Train: ego', perception['ego']['camera']),
          ('Train: infra', perception['agent']['camera'])]
show_model_on_sd(SM_ego, 'main_camera', models, idx_val=1, idx_frame=2)

In [ ]:
# show testing on infra
thresh = 0.2
perception['ego']['camera'].threshold   = thresh
perception['agent']['camera'].threshold = thresh

models = [('Train: ego', perception['ego']['camera']),
          ('Train: infra', perception['agent']['camera'])]
show_model_on_sd(SM_infra, 'CAM_INFRASTRUCTURE_001', models, idx_val=0, idx_frame=15)

In [ ]:
df = pandas.DataFrame()

In [ ]:
subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'